In [3]:
from extraOperations import PsfOtf

PsfOtf(512,0.8)


255


(<PIL.Image.Image image mode=L size=512x512 at 0x7F896C266E90>,
 array([[1.01524535e-09, 1.01529876e-09, 1.01546009e-09, ...,
         1.01572843e-09, 1.01546018e-09, 1.01529875e-09],
        [1.01529870e-09, 1.01535265e-09, 1.01551347e-09, ...,
         1.01578220e-09, 1.01551338e-09, 1.01535267e-09],
        [1.01546010e-09, 1.01551352e-09, 1.01567488e-09, ...,
         1.01594328e-09, 1.01567497e-09, 1.01551351e-09],
        ...,
        [1.01572830e-09, 1.01578228e-09, 1.01594316e-09, ...,
         1.01621201e-09, 1.01594307e-09, 1.01578229e-09],
        [1.01546010e-09, 1.01551352e-09, 1.01567488e-09, ...,
         1.01594328e-09, 1.01567497e-09, 1.01551351e-09],
        [1.01529870e-09, 1.01535265e-09, 1.01551347e-09, ...,
         1.01578220e-09, 1.01551338e-09, 1.01535267e-09]]))

# ERNet: CNN segmentation model for 

# Packages

In [8]:
import math
import os
import time 

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.autograd import Variable

from models import GetModel
from datahandler import GetDataloaders

from plotting import testAndMakeCombinedPlots


# Options

In [9]:
import argparse
opt = argparse.Namespace()

opt.root = 'trainingdata/testpartitioned_128'
opt.out = 'trained_models/ERNet_rcan-rg5-t'
opt.imageSize = 128
opt.model = 'rcan'
opt.nch_in = 3
opt.nch_out = 3 
opt.ntrain =  100
opt.ntest = 10
opt.batchSize = 3 # reduce if not enough VRAM 
opt.batchSize_test = 1
opt.n_resgroups = 15
opt.n_resblocks = 5
opt.n_feats = 48
opt.narch = 0
opt.reduction = 16
opt.lr = 0.0001 
opt.workers = 4
opt.scheduler = 10,0.5 
opt.nepoch = 1
opt.cpu = False
opt.multigpu = False
opt.log = False
opt.test = False
opt.saveinterval = 10
opt.testinterval = 1
opt.plotinterval = 33
opt.weights = ''
opt.dataset = 'pickledataset'
opt.scheduler = ''
opt.task = 'segment'
print('options loaded')

options loaded


# Train function

In [5]:
def train(dataloader, validloader, net, opt, nepoch=10):
    
    start_epoch = 0
    loss_function = nn.MSELoss()
    optimizer = optim.Adam(net.parameters(), lr=opt.lr)

    useGPU = torch.cuda.is_available() and not opt.cpu
    
    if useGPU:
        loss_function.cuda()


    if len(opt.weights) > 0: # load previous weights?
        checkpoint = torch.load(opt.weights)
        print('loading checkpoint',opt.weights)
        if opt.undomulti:
            checkpoint['state_dict'] = remove_dataparallel_wrapper(checkpoint['state_dict'])
        else:
            net.load_state_dict(checkpoint['state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer'])
            start_epoch = checkpoint['epoch']


    if len(opt.scheduler) > 0:
        stepsize, gamma = int(opt.scheduler.split(',')[0]), float(opt.scheduler.split(',')[1])
        scheduler = optim.lr_scheduler.StepLR(optimizer, stepsize, gamma=gamma, last_epoch=start_epoch-1)

    count = 0
    opt.t0 = time.perf_counter()

    for epoch in range(start_epoch, nepoch):
        mean_loss = 0

        for i, bat in enumerate(dataloader):
            lr, hr = bat[0], bat[1]
            optimizer.zero_grad()

            if useGPU:
                sr = net(lr.cuda())
                hr = hr.cuda()
            else:
                sr = net(lr)
        

            loss = loss_function(sr, hr)

            loss.backward()
            optimizer.step()
            
            
            ######### Status and display #########
            mean_loss += loss.data.item()
            print('\r[%d/%d][%d/%d] Loss: %0.6f' % (epoch+1,nepoch,i+1,len(dataloader),loss.data.item()),end='')
            
            count += 1
            if opt.log and count*opt.batchSize // 1000 > 0:
                t1 = time.perf_counter() - opt.t0
                mem = torch.cuda.memory_allocated()
                print(epoch, count*opt.batchSize, t1, mem, mean_loss / count, file=opt.train_stats)
                opt.train_stats.flush()
                count = 0



        # ---------------- Scheduler -----------------
        if len(opt.scheduler) > 0:
            scheduler.step()
            for param_group in optimizer.param_groups:
                print('\nLearning rate',param_group['lr'])
                break        


        # ---------------- Printing -----------------
        print('\nEpoch %d done, %0.6f' % (epoch,(mean_loss / len(dataloader))))
        print('\nEpoch %d done, %0.6f' % (epoch,(mean_loss / len(dataloader))),file=opt.fid)
        opt.fid.flush()
        if opt.log:
            opt.writer.add_scalar('data/mean_loss', mean_loss / len(dataloader), epoch)


        # ---------------- TEST -----------------
        if (epoch + 1) % opt.testinterval == 0:
            testAndMakeCombinedPlots(net,validloader,opt,epoch)
            # if opt.scheduler:
                # scheduler.step(mean_loss / len(dataloader))

        if (epoch + 1) % opt.saveinterval == 0:
            torch.save(net.state_dict(), opt.out + '/prelim.pth')
            checkpoint = {'epoch': epoch + 1,
            'state_dict': net.state_dict(),
            'optimizer' : optimizer.state_dict() }
            torch.save(checkpoint, opt.out + '/prelim.pth')
    
    checkpoint = {'epoch': nepoch,
    'state_dict': net.state_dict(),
    'optimizer' : optimizer.state_dict() }
    torch.save(checkpoint, opt.out + '/final.pth')



# Build Training Data

In [ ]:
%run buildTrainingData.py
# Doesn't work on google colab
# no time to figure out
# build training data on spyder

# Start training

In [ ]:
try:
    os.makedirs(opt.out)
except IOError:
    pass

opt.fid = open(opt.out + '/log.txt','w')
print(opt)
print(opt,'\n',file=opt.fid)

dataloader, validloader = GetDataloaders(opt)        
net = GetModel(opt)  

if opt.log:
    opt.train_stats = open(opt.out.replace('\\','/') + '/train_stats.csv','w')
    opt.test_stats = open(opt.out.replace('\\','/') + '/test_stats.csv','w')
    print('iter,nsample,time,memory,meanloss',file=opt.train_stats)
    print('iter,time,memory,psnr,ssim',file=opt.test_stats)

import time
t0 = time.perf_counter()
if not opt.test:
    train(dataloader, validloader, net, opt, nepoch=opt.nepoch)
else:
    if len(opt.weights) > 0: # load previous weights?
        checkpoint = torch.load(opt.weights)
        print('loading checkpoint',opt.weights)
        if opt.undomulti:
            checkpoint['state_dict'] = remove_dataparallel_wrapper(checkpoint['state_dict'])
        net.load_state_dict(checkpoint['state_dict'])
        print('time: ',time.perf_counter()-t0)
   # testAndMakeCombinedPlots(net,validloader,opt)
print('time: ',time.perf_counter()-t0)

# Tmux training

In [ ]:
!python evaluate.py --root Model_Evaluation_Edward_Tiffs/01_Network_15.04.2021_20groups_10blocks/Input_PNGs/Ting   --weights  trained_models/Previous_Models/Network_15_04_2021_20groups_10blocks --imageSize 270      --out Model_Evaluation_Edward_Tiffs/01_Network_15.04.2021_20groups_10blocks/Output_PNGs    --model rcan --nch_in 3 --nch_out 3 --n_resgroups 20  --n_resblocks 10 --workers 0

# Ed's Tiff Images

In [2]:
from skimage import io
import numpy as np
import os
from PIL import Image

filenames = ["athesim mito 1","athesim mito 3","Meng ER 1","Meng ER-3","Meng mito 2","microtubules 2"]

def Unstack():
#image dimensions of tif to numpy array are: z,y,x
#image dimensions of PNG to numpy array are: y,x, z with z being channels usually
    network1 = "01_Network_15.04.2021_20groups_10blocks"
    network2 = "02_Network_13.04.2021_15groups_5blocks"
    for j in range(0,6):
        dirTif = "Model_Evaluation_Edward_Tiffs/"   + network2 +  "/Input_Tiffs/" + filenames[j]  + ".tif"
        imTif = io.imread(dirTif)
        imTif = np.array(imTif)
        if j == 3:
            imTif = (imTif/256).astype(np.uint8)
            imTif = imTif.astype(np.uint8)
        (slides ,height, width) = imTif.shape 

        for i in range(0,slides) : #TIFslides is not included, only goes up to TIFslides - 1
             slide = np.copy(imTif[i,:,:])
             slide = np.expand_dims(slide, axis = 2)
             slide = np.concatenate((slide,slide,slide), axis=2)
             slide = slide.astype(np.uint8)
             slide = Image.fromarray(slide) 
             slide_name = "Model_Evaluation_Edward_Tiffs/"  + network2 + "/Input_PNGs/" + filenames[j] + "_slide_" + str(i+1) + ".png"
             slide.save(slide_name)
Unstack()

In [ ]:
!python evaluate.py --root  Model_Evaluation_Edward_Tiffs/02_Network_13.04.2021_15groups_5blocks/Input_PNGs   --weights  trained_models/Previous_Models/Network_13_04_2021_15groups_5blocks --imageSize 270             --out Model_Evaluation_Edward_Tiffs/02_Network_13.04.2021_15groups_5blocks/Output_PNGs    --model rcan --nch_in 3 --nch_out 3 --n_resgroups 15  --n_resblocks 5 --workers 0

In [4]:
from skimage import io
import numpy as np
from PIL import Image
filenames = ["athesim mito 1","athesim mito 3","Meng ER 1","Meng ER-3","Meng mito 2","microtubules 2"]

def Restack():
    network1 = "01_Network_15.04.2021_20groups_10blocks"
    network2 = "02_Network_13.04.2021_15groups_5blocks"
    for j in range(0,6):
        dirTif = "Model_Evaluation_Edward_Tiffs/"   + network2 +  "/Input_Tiffs/" + filenames[j]  + ".tif"
        imTif = io.imread(dirTif)
        imTif = np.array(imTif)
        if j == 3:
            imTif = (imTif/256).astype(np.uint8)
            imTif = imTif.astype(np.uint8)
        (slides ,height, width) = imTif.shape 
        newTif = imTif
        for i in range(0,slides) : #TIFslides is not included, only goes up to TIFslides - 1
             dirPNG = "Model_Evaluation_Edward_Tiffs/" + network2 + "/Output_PNGs/" + filenames[j] +  "_slide_" + str(i+1) + "_out.png"
             slide = io.imread(dirPNG)
             slide = np.array(slide)
             slide = np.mean(slide,axis = 2)
             newTif[i,:,:] = slide   
        newTif_dir =  "Model_Evaluation_Edward_Tiffs/"   + network2 +  "/Output_Tiffs/" + filenames[j]  + "_out.tif"
        io.imsave(newTif_dir, newTif)
Restack()
 

# Evaluate the model

In [10]:
import glob
from PIL import Image
import random
import parser
import os
import scipy.ndimage as ndimage
import numpy as np
import pickle
import cv2
from skimage.measure import compare_ssim
from skimage import io
from extraOperations import PSFnoise

number_of_raw_inputs = 5
raw_inputs = ['trained_models/Previous_Models/Network_Bovey_Widefield_Restoration/Processed_Input_A/001.png', 'trained_models/Previous_Models/Network_Bovey_Widefield_Restoration/Processed_Input_A/002.png','trained_models/Previous_Models/Network_Bovey_Widefield_Restoration/Processed_Input_B/003.png','trained_models/Previous_Models/Network_Bovey_Widefield_Restoration/Processed_Input_B/004.png','trained_models/Previous_Models/Network_Bovey_Widefield_Restoration/Processed_Input_B/005.png']

    
for i in range (0,number_of_raw_inputs,1):   # list indexing starts at 0
    image = Image.open(raw_inputs[i])
    image = np.array(image)
    image = np.mean(image,axis = 2)
    image = image.astype(np.uint8)
    image = Image.fromarray(image)
    dir = 'trained_models/Previous_Models/Network_Bovey_Widefield_Restoration/Model_evaluation_ouput' + '/00' + str(i) + "_out.png"
    image.save(dir)


#    if (i < 2):
 #       image = np.expand_dims(image, axis = 2)
  #      image = np.concatenate((image,image,image), axis=2)
   #     image = Image.fromarray(image)
    #    image_name = "Model_Evaluation_Input/Processed_Input_A/00" + str(i+1) +  ".png"
#        image.save(image_name)
 #   else:
  #      image = Image.fromarray(image)
   #     image_name = "Model_Evaluation_Input/Processed_Input_B/00" + str(i+1) +  ".png"
    #    image.save(image_name)

In [2]:
!python evaluate.py --root trained_models/Previous_Models/Network_Bovey_Widefield_Restoration/Processed_Input_B --weights trained_models/Previous_Models/Network_Bovey_Widefield_Restoration --imageSize 750 --out trained_models/Previous_Models/Network_Bovey_Widefield_Restoration/Model_evaluation_ouput --model rcan --nch_in 1 --nch_out 1 --n_resgroups 18 --n_resblocks 8

Namespace(batchSize=16, batchSize_test=1, cpu=False, dataset='imagedataset', fid=<_io.TextIOWrapper name='trained_models/Previous_Models/Network_Bovey_Widefield_Restoration/Model_evaluation_ouput/log.txt' mode='w' encoding='UTF-8'>, imageSize=750, log=False, lr=0.0001, model='rcan', multigpu=False, n_feats=64, n_resblocks=8, n_resgroups=18, narch=0, nch_in=1, nch_out=1, nepoch=10, ntest=10, ntrain=0, out='trained_models/Previous_Models/Network_Bovey_Widefield_Restoration/Model_evaluation_ouput', plotinterval=1, reduction=16, root='trained_models/Previous_Models/Network_Bovey_Widefield_Restoration/Processed_Input_B', saveinterval=10, scheduler='', test=False, testinterval=1, undomulti=False, weights='trained_models/Previous_Models/Network_Bovey_Widefield_Restoration/final.pth', workers=6)
loading checkpoint trained_models/Previous_Models/Network_Bovey_Widefield_Restoration/final.pth
0.0 1.0 (1040, 1392)
[4/4][1/3], shape is 750x750 - trained_models/Previous_Models/Network_Bovey_Widefiel

# Evaluating Triplets

In [ ]:
from buildTrainingData import noisy
from PIL import Image
import numpy as np
from skimage import io

filename = "Model_Evaluation_Triplets/0002.png"
img = Image.open(filename)
img = np.array(img.mean)
height = img.shape[0]
img = img[700:1212,700:1212,:]
io.imsave(filename.replace(".png","_gt.png"), img)

#Copy of Poisson noise code which is actually PSF noise, followed by Gaussian noise
poisson_param = 100 
img = img.astype(float) 
img = noisy("poisson",img,[0,poisson_param])
gauss_param = abs(0.002*np.random.randn()+ 0.005)
print("got to this point")
img = noisy("gauss",img,[0,gauss_param])

img = img.astype("uint8")
io.imsave(filename.replace(".png","_degraded.png"),img)
print("degraded image savded")


!python evaluate.py --root Model_Evaluation_Triplets/Degraded --weights  trained_models/Previous_Models/Network_15.04.2021_20groups_10blocks --imageSize 260 --out Model_Evaluation_Triplets/Processed  --model rcan --nch_in 3 --nch_out 3 --n_resgroups 20  --n_resblocks 10 --workers 0

In [ ]:
!python run.py --root trainingdata/testpartitioned_128 --out trained_models/ERNet_rcan-rg5-t --imageSize 128 --model rcan --nch_in 3 --nch_out 3 --ntrain 1500 --ntest 100 --batchSize 3 --n_resgroups 20 --n_resblocks 10 --lr 0.0001 --scheduler 20,0.5  --plotinterval 33 --dataset pickledataset

In [ ]:
!python evaluate.py --root Model_Evaluation_Input/Processed_Input_B --weights trained_models/ERNet_rcan-rg5-t/ --imageSize 800   --out Model_Evaluation_Output/ --model rcan --nch_in 3 --nch_out 3 --n_resgroups 20  --n_resblocks 10 

# Generate Bovey Training Data

In [16]:
import os
from PIL import Image
import numpy as np

rootdir = "trainingdata/testBovey1024/oosker"
files = [x[2] for x in os.walk(rootdir)]
print(files[0])
files = files[0]
for i in range(0,len(files)):
    file_name = rootdir + "/" + files[i]
    file_name2 = file_name.replace("oosker","oosker_greyscale/")
    file_name3 = file_name2.replace(".png","_greyscale.png")
    image = Image.open(file_name)
    image = np.array(image)
    image = np.mean(image, axis=2)
    image = image.astype(np.uint8)
    image = Image.fromarray(image)
    image.save(file_name3)




#subdirs = [x[0] for x in os.walk(rootdir)]
#print(subdirs)
#for i in range(1,len(subdirs)):
#    subdirs2 = [x[2] for x in os.walk(subdirs[i])]
#    subdirs2 = subdirs2[0]
#    for j in range(0,len(subdirs2)):
#        file_name = subdirs[i] +"/"+ subdirs2[j]
#        file_name2 = subdirs[i] +"_"+ subdirs2[j]
#        file_name3 = file_name2.replace("oosker","oosker_greyscale")
#        image = Image.open(file_name)  
#        image.save(file_name3)
        
#print(subdirs2)
#for subdir, dirs, files in os.walk(rootdir): in subdirs
 #   for subdir in subdirs:


['20_1-0_testimg.png', '20_5-0_testimg.png', '4_2-0_testimg.png', '20_2-0_testimg.png', '12_6-0_testimg.png', '4_1-0_testimg.png', '20_0-0_testimg.png', '12_2-0_testimg.png', '16_2-0_testimg.png', '16_3-0_testimg.png', '16_1-0_testimg.png', '12_0-0_testimg.png', '12_5-0_testimg.png', '12_4-0_testimg.png', '1_6-0_testimg.png', '16_5-0_testimg.png', '8_4-0_testimg.png', '4_6-0_testimg.png', '8_0-0_testimg.png', '4_5-0_testimg.png', '12_1-0_testimg.png', '1_2-0_testimg.png', '8_6-0_testimg.png', '16_0-0_testimg.png', '4_4-0_testimg.png', '8_2-0_testimg.png', '8_1-0_testimg.png', '12_3-0_testimg.png', '8_3-0_testimg.png', '20_6-0_testimg.png', '16_6-0_testimg.png', '20_4-0_testimg.png', '1_5-0_testimg.png', '16_4-0_testimg.png', '1_4-0_testimg.png', '4_3-0_testimg.png', '1_1-0_testimg.png', '4_0-0_testimg.png', '20_3-0_testimg.png', '8_5-0_testimg.png', '1_3-0_testimg.png', '1_0-0_testimg.png']


### Oscar 15g5b

In [ ]:
!python evaluate.py --root  trainingdata/testBovey1024/oosker   --weights  trained_models/Previous_Models/Network_13_04_2021_15groups_5blocks    --imageSize 260            --out trainingdata/testBovey1024/oosker15g5b   --model rcan --nch_in 3 --nch_out 3 --n_resgroups 15  --n_resblocks 5 --workers 0


trainingdata/testBovey1024/oosker

### Oscar 20g10b

In [ ]:
!python evaluate.py --root  trainingdata/testBovey1024/oosker   --weights  trained_models/Previous_Models/Network_15_04_2021_20groups_10blocks --imageSize 260             --out trainingdata/testBovey1024/oosker20g10b    --model rcan --nch_in 3 --nch_out 3 --n_resgroups 15  --n_resblocks 5 --workers 0

### Bovey denoising

In [1]:
!python evaluate.py --root  trainingdata/testBovey1024/oosker_greyscale  --weights  trained_models/Previous_Models/Network_Bovey_Widefield_Restoration  --imageSize 270             --out trainingdata/testBovey1024/ooskerBovey    --model rcan --nch_in 1 --nch_out 1 --n_resgroups 18  --n_resblocks 8 --workers 0

Namespace(batchSize=16, batchSize_test=1, cpu=False, dataset='imagedataset', fid=<_io.TextIOWrapper name='trainingdata/testBovey1024/ooskerBovey/log.txt' mode='w' encoding='UTF-8'>, imageSize=270, log=False, lr=0.0001, model='rcan', multigpu=False, n_feats=64, n_resblocks=8, n_resgroups=18, narch=0, nch_in=1, nch_out=1, nepoch=10, ntest=10, ntrain=0, out='trainingdata/testBovey1024/ooskerBovey', plotinterval=1, reduction=16, root='trainingdata/testBovey1024/oosker_greyscale', saveinterval=10, scheduler='', test=False, testinterval=1, undomulti=False, weights='trained_models/Previous_Models/Network_Bovey_Widefield_Restoration/final.pth', workers=0)
loading checkpoint trained_models/Previous_Models/Network_Bovey_Widefield_Restoration/final.pth
0.0 0.9607843137254902 (512, 512)
[4/4][1/42], shape is 270x270 - trainingdata/testBovey1024/oosker_greyscale/16_0-0_testimg_greyscale.png 0
saved
0.0 1.0 (512, 512)
[4/4][2/42], shape is 270x270 - trainingdata/testBovey1024/oosker_greyscale/12_1-0